In [ ]:
#instructions for how to build this using nbdev at https://nbdev.fast.ai/

In [ ]:
#default_exp visualize

# Predict loaded articles

> Takes a loader as an argument

In [ ]:
#export
from newstrends import loader, describe
import pandas as pd
import datetime, os
from datetime import timezone
import matplotlib.pyplot as plt

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
class visualizer:
    "my visualizer! Don't know what I'm doing with it yet though"
    
    def runDefault(self, outdir="."):
        
        publisherList = ["newyorktimes", "washingtonpost"]
        
        describer = describe.describer()
        
        describer.set_articleDir(path="../CoverageTrends")
        describer.load_articles(publications=publisherList, lastN=5)
        describer.fitVectorizer(ngram_range=(1,1))
        
        topN = pd.DataFrame()
        
        for publisher in publisherList:
            topN[publisher] = [x[0] for x in describer.getTopNWords(10, source=[publisher])]
            
        vcs = topN.melt(var_name='publisher', value_name='words')["words"].value_counts()
        myTime = datetime.datetime.now(tz=timezone.utc).strftime('%Y%m%d-%H%M')
        myTime = myTime[:-1]
        myTime +="0"
        plt.close('all') #in case of zombies or something
        os.makedirs("{}/img".format(outdir), exist_ok=True)
        os.makedirs("{}/timeseries".format(outdir), exist_ok=True)

        df = describer.df
                
        for middleWord in vcs.dropna().index: #k, this is going to be wayyy too many images, but just testing

            tmp = df[df["tokens"].apply(lambda x: (middleWord in x))].copy().fillna(0)
            tmp.date = pd.to_datetime(tmp.date)
            tmp = tmp.groupby(["source", "date"]).count()["quickReplace"]
            try:  #for some reason, sometimes the formatting's getting messed up
                tmp.unstack(level=0).fillna(0).to_pickle("{}/timeseries/{}.pkl".format(outdir, middleWord))
            except:
                pass
            ax = tmp.unstack(level=0).fillna(0).plot(title="Frontpage mentions of {}".format(middleWord), figsize=(8,8))
            ax.set_ylabel("frontpage mentions at time")
            try:
                deleteMe = [oldFile for oldFile in os.listdir("{}/img".format(outdir)) if oldFile.endswith(middleWord+".jpg")]
                for oldFile in deleteMe:
                    os.remove("{}/img/{}".format(outdir, oldFile))
            except:
                pass

            ax.figure.savefig("{}/img/{}_{}.jpg".format(outdir, myTime, middleWord))
            plt.close('all') #close all figures


In [ ]:
test = visualizer()

In [ ]:
test.runDefault()